In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import numpy as np
import pickle
import json
import pandas as pd
from datetime import datetime
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
from nilearn.plotting import plot_design_matrix
from nilearn.image import load_img
from nilearn.reporting import make_glm_report
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
sys.path.append('..')
from utils.data import Subject, create_dummy_regressors
from utils.analysis import compute_parametric_modulator

In [3]:
base_dir = '/home/ubuntu/data/learning-habits'
bids_dir = "/home/ubuntu/data/learning-habits/bids_dataset/derivatives/fmriprep-24.0.1"

In [4]:
model_params = {
    'model_name': 'non_param_both',
    'tr': 2.33384,
    'hrf_model': 'spm',
    'noise_model': 'ar1',
    'smoothing_fwhm': 5,
    'high_pass': 0.01,
    'motion_type': 'basic',
    'fd_thresh': 0.5,
    'std_dvars_thresh': 2.5,
    'scrub': 'dummies',
    'modulators': 'non_parametric',
    'modulator_normalization': None,
    'exclude_stimuli': True,
    'include_physio': True,
    'brain_mask': True,
}

run = 'test'

In [5]:
subject = Subject(base_dir, '01', include_modeling=True, include_imaging=True, bids_dir=bids_dir)

In [6]:
# Parameters
model_name = model_params["model_name"]
tr = model_params["tr"]
hrf_model = model_params["hrf_model"]
noise_model = model_params["noise_model"]
smoothing_fwhm = model_params["smoothing_fwhm"]
high_pass = model_params["high_pass"]
include_physio = model_params["include_physio"]
brain_mask = model_params["brain_mask"]
modulators = model_params["modulators"]
modulator_normalization = model_params["modulator_normalization"]
exclude_stimuli = model_params["exclude_stimuli"]
motion_type = model_params["motion_type"]
fd_thresh = model_params["fd_thresh"]
std_dvars_thresh = model_params["std_dvars_thresh"]
scrub = model_params["scrub"]

In [7]:
# Load confounds
confounds, sample_mask = subject.load_confounds(run, motion_type=motion_type,
                                                fd_thresh=fd_thresh, std_dvars_thresh=std_dvars_thresh,
                                                scrub=(0 if scrub == 'dummies' else scrub))
if include_physio:
    physio_regressors = subject.load_physio_regressors(run)
    confounds = confounds.join(physio_regressors)

In [8]:
if scrub == 'dummies':
    dummies = create_dummy_regressors(sample_mask, len(confounds))
    confounds = pd.concat([confounds, dummies], axis=1)

In [9]:
# Load fMRI volume
img_path = subject.img.get(run)
fmri_img = load_img(img_path)

In [10]:
if brain_mask:
    brain_mask_path = subject.brain_mask.get(run)
    brain_mask = load_img(brain_mask_path)
else:
    brain_mask = None

n = fmri_img.shape[-1]
frametimes = np.linspace(tr / 2., (n - .5) * tr, n)

In [11]:
stim_rewards = {0: 0, 1: 1, 2: 2, 3: 2, 4: 3, 5: 3, 6: 4, 7: 4, 8: 5}
stim_frequ = {0: 0, 1: 0, 2: -1, 3: 1, 4: -1, 5: 1, 6: -1, 7: 1, 8: 0}

In [12]:
block = getattr(subject, run)

In [13]:
columns_event = {'first_stim':'first_stim_presentation',
                 'first_stim_value_rl':'first_stim_presentation',
                'first_stim_value_ck':'first_stim_presentation'}
events = block.extend_events_df(columns_event)

In [14]:
if exclude_stimuli:
    events['trial_type'] = events.apply(
        lambda row: f"{row['trial_type']}_{'exclude' if int(row['first_stim']) in (1, 8) else 'include'}"
        if row['trial_type'] == 'first_stim_presentation' else row['trial_type'],
        axis=1
    )

In [15]:
events['first_stim'] = events['first_stim'].astype(int)
events['first_stim_reward'] = events['first_stim'].map(stim_rewards)
events['first_stim_frequ'] = events['first_stim'].map(stim_frequ)

In [16]:
events.head(15)

,onset,duration,trial_type,trial,first_stim,first_stim_value_rl,first_stim_value_ck,first_stim_reward,first_stim_frequ
0,0.002346,0.813894,first_stim_presentation_exclude,1,1,1.000017,0.042455,1,0
1,0.816239,0.550671,second_stim_presentation,1,0,0.000000,0.000000,0,0
2,1.366910,0.000000,response,1,0,0.000000,0.000000,0,0
3,1.370405,1.951056,purple_frame,1,0,0.000000,0.000000,0,0
4,3.321461,6.203388,iti,1,0,0.000000,0.000000,0,0
5,9.524849,0.805284,first_stim_presentation_exclude,2,1,1.000017,0.042166,1,0
6,10.330133,0.443547,second_stim_presentation,2,0,0.000000,0.000000,0,0
7,10.773680,0.000000,response,2,0,0.000000,0.000000,0,0
8,10.776506,2.057924,purple_frame,2,0,0.000000,0.000000,0,0
9,12.834430,6.499065,iti,2,0,0.000000,0.000000,0,0


In [17]:
events.first_stim_reward.unique()

array([1, 0, 2, 5, 4, 3])

In [18]:
# Ignore warnings related to null duration events and unexpected columns in events data
warnings.filterwarnings("ignore", message=".*events with null duration.*")
warnings.filterwarnings("ignore", message=".*following unexpected columns in events data.*")


# Create design matrix
design_matrix = make_first_level_design_matrix(frame_times=frametimes,
                                    events=events,
                                    hrf_model=hrf_model,
                                    drift_model=None,
                                    high_pass=high_pass,
                                    add_regs=confounds)


modulator_normalization = None
# RL Parametric modulation
parametric_modulator_column = 'first_stim_reward'
condition = 'first_stim_presentation_include' if exclude_stimuli else 'first_stim_presentation'
reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                            frametimes, hrf_model, normalize=modulator_normalization)
design_matrix.insert(2, parametric_modulator_column, reg_value)

# CK Parametric modulation
parametric_modulator_column = 'first_stim_frequ'
condition = 'first_stim_presentation_include' if exclude_stimuli else 'first_stim_presentation'
reg_value = compute_parametric_modulator(events, condition, parametric_modulator_column,
                                            frametimes, hrf_model, normalize=modulator_normalization)
design_matrix.insert(3, parametric_modulator_column, reg_value)

In [19]:
design_matrix.head(20)

,first_stim_presentation_exclude,first_stim_presentation_include,first_stim_reward,first_stim_frequ,iti,non_response_feedback,purple_frame,response,second_stim_presentation,csf,...,scrub_vol_566,scrub_vol_572,scrub_vol_577,scrub_vol_578,scrub_vol_580,scrub_vol_581,scrub_vol_583,scrub_vol_584,scrub_vol_588,constant
1.16692,0.001193,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.000001,7.528526,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3.50076,0.098618,0.000000,0.000000,0.000000,6.874239e-09,0.0,0.023896,0.002221,0.039245,19.601565,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5.83460,0.164194,0.000000,0.000000,0.000000,4.734032e-02,0.0,0.293430,0.009459,0.116637,-7.243666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8.16844,0.095794,0.000000,0.000000,0.000000,4.181130e-01,0.0,0.382594,0.007674,0.084576,16.888935,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10.50228,0.030105,0.000000,0.000000,0.000000,8.581891e-01,0.0,0.204721,0.003109,0.031752,6.986960,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12.83612,0.083943,0.000000,0.000000,0.000000,9.515173e-01,0.0,0.076025,0.002505,0.028311,-5.371024,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
15.16996,0.151816,0.000000,0.000000,0.000000,5.947070e-01,0.0,0.274158,0.008736,0.078111,-8.671603,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17.50380,0.088517,0.000000,0.000000,0.000000,5.433253e-01,0.0,0.370786,0.006895,0.055297,29.183064,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19.83764,0.024171,0.000010,0.000019,0.000010,7.941654e-01,0.0,0.194094,0.002510,0.017725,-6.053134,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22.17148,-0.006243,0.059185,0.118371,0.059185,9.003217e-01,0.0,0.049825,0.000784,0.010977,32.695102,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
